In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
import random
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import os
import tempfile

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11242664112583173373
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 1734816564
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 10343484271954322518
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
 xla_global_id: 416903419]

In [3]:
def reduce_mem(df: pd.DataFrame):
    "This method reduces memory for numeric columns in the dataframe";

    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64', "uint16", "uint32", "uint64"];
    start_mem = df.memory_usage().sum() / 1024**2;

    for col in df.columns:
        col_type = df[col].dtypes

        if col_type in numerics:
            c_min = df[col].min();
            c_max = df[col].max();

            if "int" in str(col_type):
                if c_min >= np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min >= np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min >= np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min >= np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min >= np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                if c_min >= np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)  

    end_mem = df.memory_usage().sum() / 1024**2

    print(f"Start - end memory:- {start_mem:5.2f} - {end_mem:5.2f} Mb");
    return df;

In [4]:
def my_seed_everywhere(seed: int = 42):
    random.seed(seed) # random
    np.random.seed(seed) # np
    os.environ["PYTHONHASHSEED"] = str(seed) # os
    tf.random.set_seed(seed) # tensorflow

my_seed = 42
my_seed_everywhere(my_seed)

In [6]:
train = pd.read_csv("./playground-series-s4e7/train.csv")
target = pd.read_csv("./playground-series-s4e7/test.csv")

In [7]:
neg, pos = np.bincount(train['Response'])
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))

Examples:
    Total: 11504798
    Positive: 1415059 (12.30% of total)



In [8]:
category_columns = train.select_dtypes(include="object").columns
category_columns = category_columns.tolist()

def label_encoding(df):
    # Instance of LabelEncoder
    label_encoders = {col: LabelEncoder() for col in category_columns}

    # Label Encoding
    for col in category_columns:
        df[col] = label_encoders[col].fit_transform(df[col])

label_encoding(train)

In [9]:
category_columns = target.select_dtypes(include="object").columns
category_columns = category_columns.tolist()

def label_encoding(df):
    # Instance of LabelEncoder
    label_encoders = {col: LabelEncoder() for col in category_columns}

    # Label Encoding
    for col in category_columns:
        df[col] = label_encoders[col].fit_transform(df[col])

label_encoding(target)

In [10]:
train = train.drop(columns=['id'])
target = target.drop(columns=['id'])

In [11]:
target

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,0,20,1,47.0,0,1,0,2630.0,160.0,228
1,1,47,1,28.0,0,0,1,37483.0,124.0,123
2,1,47,1,43.0,0,0,1,2630.0,26.0,271
3,0,22,1,47.0,1,1,0,24502.0,152.0,115
4,1,51,1,19.0,0,0,0,34115.0,124.0,148
...,...,...,...,...,...,...,...,...,...,...
7669861,1,57,1,28.0,0,0,1,51661.0,124.0,109
7669862,1,28,1,50.0,1,1,0,25651.0,152.0,184
7669863,1,47,1,33.0,1,0,0,2630.0,138.0,63
7669864,1,30,1,28.0,0,1,1,38866.0,124.0,119


In [12]:
train = reduce_mem(train)
target = reduce_mem(target)

Start - end memory:- 833.86 - 230.41 Mb
Start - end memory:- 497.39 - 146.29 Mb


In [13]:
# Use a utility from sklearn to split and shuffle your dataset.
train_df, test_df = train_test_split(train, test_size=0.2)
train_df, val_df = train_test_split(train_df, test_size=0.2)

# Form np arrays of labels and features.
train_labels = np.array(train_df.pop('Response'))
bool_train_labels = train_labels != 0
val_labels = np.array(val_df.pop('Response'))
test_labels = np.array(test_df.pop('Response'))

train_features = np.array(train_df)
val_features = np.array(val_df)
test_features = np.array(test_df)

In [14]:
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)

val_features = scaler.transform(val_features)
test_features = scaler.transform(test_features)
target_features = scaler.transform(target)

train_features = np.clip(train_features, -5, 5)
val_features = np.clip(val_features, -5, 5)
test_features = np.clip(test_features, -5, 5)


print('Training labels shape:', train_labels.shape)
print('Validation labels shape:', val_labels.shape)
print('Test labels shape:', test_labels.shape)

print('Training features shape:', train_features.shape)
print('Validation features shape:', val_features.shape)
print('Test features shape:', test_features.shape)
print('Target features shape:', target_features.shape)

C:\Users\OSY\anaconda3\envs\osy\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


Training labels shape: (7363070,)
Validation labels shape: (1840768,)
Test labels shape: (2300960,)
Training features shape: (7363070, 10)
Validation features shape: (1840768, 10)
Test features shape: (2300960, 10)
Target features shape: (7669866, 10)


In [15]:
# Define the metrics
METRICS = [
    keras.metrics.TruePositives(name='tp'),
    keras.metrics.FalsePositives(name='fp'),
    keras.metrics.TrueNegatives(name='tn'),
    keras.metrics.FalseNegatives(name='fn'),
    keras.metrics.BinaryAccuracy(name='accuracy'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
    keras.metrics.AUC(name='auc'),
    keras.metrics.AUC(name='prc', curve='PR'),  # precision-recall curve
]
def make_model(metrics=METRICS, output_bias=None):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
    
    model = keras.Sequential([
    keras.layers.Dense(16, activation='relu', input_shape=(train_features.shape[-1],)),
    
    # 두 번째 은닉층: 12개의 뉴런, ReLU 활성화 함수
    keras.layers.Dense(12, activation='relu'),
    
    # 드롭아웃 레이어: 50%의 뉴런을 랜덤하게 비활성화
    keras.layers.Dropout(0.5),
    
    # 세 번째 은닉층: 8개의 뉴런, ReLU 활성화 함수
    keras.layers.Dense(8, activation='relu'),
    
    # 출력층: 1개의 뉴런, 시그모이드 활성화 함수, 바이어스 초기화
    keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias),
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=metrics
    )

    return model

In [16]:
EPOCHS = 100
BATCH_SIZE = 2048

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_prc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

In [17]:
pos_features = train_features[bool_train_labels]
neg_features = train_features[~bool_train_labels]

pos_labels = train_labels[bool_train_labels]
neg_labels = train_labels[~bool_train_labels]

In [18]:
ids = np.arange(len(pos_features))
choices = np.random.choice(ids, len(neg_features))

res_pos_features = pos_features[choices]
res_pos_labels = pos_labels[choices]

res_pos_features.shape

(6457618, 10)

In [19]:
resampled_features = np.concatenate([res_pos_features, neg_features], axis=0)
resampled_labels = np.concatenate([res_pos_labels, neg_labels], axis=0)

order = np.arange(len(resampled_labels))
np.random.shuffle(order)
resampled_features = resampled_features[order]
resampled_labels = resampled_labels[order]

resampled_features.shape

(12915236, 10)

In [20]:
BUFFER_SIZE = 100000

def make_ds(features, labels):
    ds = tf.data.Dataset.from_tensor_slices((features, labels))#.cache()
    ds = ds.shuffle(BUFFER_SIZE).repeat()
    return ds

pos_ds = make_ds(pos_features, pos_labels)
neg_ds = make_ds(neg_features, neg_labels)

In [21]:
for features, label in pos_ds.take(1):
    print("Features:\n", features.numpy())
    print()
    print("Label: ", label.numpy())

Features:
 [ 0.9202957   0.57515174  0.04449861  0.27560756 -0.9281243  -0.9046776
  0.99428636 -1.6898099   0.21376617 -1.4619182 ]

Label:  1


In [22]:
resampled_ds = tf.data.Dataset.sample_from_datasets([pos_ds, neg_ds], weights=[0.5, 0.5])
resampled_ds = resampled_ds.batch(BATCH_SIZE).prefetch(2)

In [23]:
for features, label in resampled_ds.take(1):
    print(label.numpy().mean())

0.50830078125


In [24]:
resampled_steps_per_epoch = np.ceil(2.0*neg/BATCH_SIZE)
resampled_steps_per_epoch

9854.0

In [25]:
resampled_model = make_model()

In [26]:
resampled_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                176       
                                                                 
 dense_1 (Dense)             (None, 12)                204       
                                                                 
 dropout (Dropout)           (None, 12)                0         
                                                                 
 dense_2 (Dense)             (None, 8)                 104       
                                                                 
 dense_3 (Dense)             (None, 1)                 9         
                                                                 
Total params: 493
Trainable params: 493
Non-trainable params: 0
_________________________________________________________________


In [27]:
resampled_model.predict(train_features[:10])

1/1 [==============================] - 2s 2s/step


array([[0.88937527],
       [0.8931116 ],
       [0.90248084],
       [0.94258714],
       [0.87929577],
       [0.8821278 ],
       [0.88500327],
       [0.9293072 ],
       [0.88992506],
       [0.92086816]], dtype=float32)

In [28]:
results = resampled_model.evaluate(train_features, train_labels, batch_size=BATCH_SIZE, verbose=0)
print("Loss: {:0.4f}".format(results[0]))

Loss: 2.1216


In [29]:
initial_bias = np.log([pos/neg])
initial_bias

array([-1.96434774])

In [30]:
resampled_model = make_model(output_bias=initial_bias)
resampled_model.predict(train_features[:10])

1/1 [==============================] - 0s 77ms/step


array([[0.17884915],
       [0.14621922],
       [0.17297141],
       [0.21432428],
       [0.25864837],
       [0.15943715],
       [0.16320243],
       [0.1559958 ],
       [0.19637148],
       [0.3861751 ]], dtype=float32)

In [31]:
results = resampled_model.evaluate(train_features, train_labels, batch_size=BATCH_SIZE, verbose=0)
print("Loss: {:0.4f}".format(results[0]))

Loss: 0.4105


In [33]:
initial_weights = os.path.join(tempfile.mkdtemp(), 'initial_weights')
resampled_model.save_weights(initial_weights)

In [34]:
weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

Weight for class 0: 0.57
Weight for class 1: 4.07


In [35]:
with tf.device('/GPU:0'):
    resampled_model.load_weights(initial_weights)

    # Reset the bias to zero, since this dataset is balanced.
    output_layer = resampled_model.layers[-1] 
    output_layer.bias.assign([0])

    val_ds = tf.data.Dataset.from_tensor_slices((val_features, val_labels)).cache()
    val_ds = val_ds.batch(BATCH_SIZE).prefetch(2) 

    resampled_history = resampled_model.fit(
        resampled_ds,
        epochs=EPOCHS,
        steps_per_epoch=resampled_steps_per_epoch,
        callbacks=[early_stopping],
        validation_data=val_ds)

Epoch 1/100
9854/9854 [==============================] - 133s 13ms/step - loss: 0.4331 - tp: 9278247.0000 - fp: 3406408.0000 - tn: 6683918.0000 - fn: 812419.0000 - accuracy: 0.7910 - precision: 0.7315 - recall: 0.9195 - auc: 0.8480 - prc: 0.7890 - val_loss: 0.3567 - val_tp: 186289.0000 - val_fp: 405617.0000 - val_tn: 1208728.0000 - val_fn: 40134.0000 - val_accuracy: 0.7578 - val_precision: 0.3147 - val_recall: 0.8227 - val_auc: 0.8561 - val_prc: 0.3647
Epoch 2/100
9854/9854 [==============================] - 133s 14ms/step - loss: 0.4242 - tp: 9201900.0000 - fp: 3230227.0000 - tn: 6858428.0000 - fn: 890437.0000 - accuracy: 0.7958 - precision: 0.7402 - recall: 0.9118 - auc: 0.8535 - prc: 0.7966 - val_loss: 0.3415 - val_tp: 152953.0000 - val_fp: 296468.0000 - val_tn: 1317877.0000 - val_fn: 73470.0000 - val_accuracy: 0.7990 - val_precision: 0.3403 - val_recall: 0.6755 - val_auc: 0.8565 - val_prc: 0.3656
Epoch 3/100
 681/9854 [=>............................] - ETA: 2:01 - loss: 0.4246 - tp

KeyboardInterrupt: 

In [40]:
with tf.device('/GPU:0'):
    resampled_model = make_model()
    resampled_model.load_weights(initial_weights)

    # Reset the bias to zero, since this dataset is balanced.
    output_layer = resampled_model.layers[-1] 
    output_layer.bias.assign([0])

    resampled_history = resampled_model.fit(
        resampled_ds,
        # These are not real epochs
        steps_per_epoch=20,
        epochs=EPOCHS,
        validation_data=(val_ds))

Epoch 1/100
20/20 [==============================] - 7s 271ms/step - loss: 0.7581 - tp: 28029.0000 - fp: 24190.0000 - tn: 6460.0000 - fn: 2761.0000 - accuracy: 0.5613 - precision: 0.5368 - recall: 0.9103 - auc: 0.5699 - prc: 0.5282 - val_loss: 0.8158 - val_tp: 219444.0000 - val_fp: 1515594.0000 - val_tn: 98751.0000 - val_fn: 6979.0000 - val_accuracy: 0.1729 - val_precision: 0.1265 - val_recall: 0.9692 - val_auc: 0.5492 - val_prc: 0.1303
Epoch 2/100
20/20 [==============================] - 5s 256ms/step - loss: 0.7030 - tp: 18527.0000 - fp: 17281.0000 - tn: 3038.0000 - fn: 2114.0000 - accuracy: 0.5265 - precision: 0.5174 - recall: 0.8976 - auc: 0.5543 - prc: 0.5364 - val_loss: 0.7491 - val_tp: 209686.0000 - val_fp: 1256728.0000 - val_tn: 357617.0000 - val_fn: 16737.0000 - val_accuracy: 0.3082 - val_precision: 0.1430 - val_recall: 0.9261 - val_auc: 0.6470 - val_prc: 0.1745
Epoch 3/100
20/20 [==============================] - 5s 240ms/step - loss: 0.6796 - tp: 17604.0000 - fp: 14887.0000 

20/20 [==============================] - 5s 256ms/step - loss: 0.4626 - tp: 18536.0000 - fp: 7163.0000 - tn: 13141.0000 - fn: 2120.0000 - accuracy: 0.7734 - precision: 0.7213 - recall: 0.8974 - auc: 0.8224 - prc: 0.7530 - val_loss: 0.3964 - val_tp: 215653.0000 - val_fp: 586648.0000 - val_tn: 1027697.0000 - val_fn: 10770.0000 - val_accuracy: 0.6755 - val_precision: 0.2688 - val_recall: 0.9524 - val_auc: 0.8397 - val_prc: 0.3175
Epoch 20/100
20/20 [==============================] - 5s 249ms/step - loss: 0.4632 - tp: 18492.0000 - fp: 7233.0000 - tn: 13078.0000 - fn: 2157.0000 - accuracy: 0.7708 - precision: 0.7188 - recall: 0.8955 - auc: 0.8230 - prc: 0.7580 - val_loss: 0.3933 - val_tp: 216051.0000 - val_fp: 587995.0000 - val_tn: 1026350.0000 - val_fn: 10372.0000 - val_accuracy: 0.6749 - val_precision: 0.2687 - val_recall: 0.9542 - val_auc: 0.8402 - val_prc: 0.3178
Epoch 21/100
20/20 [==============================] - 5s 249ms/step - loss: 0.4638 - tp: 18429.0000 - fp: 7380.0000 - tn: 130

20/20 [==============================] - 5s 252ms/step - loss: 0.4418 - tp: 18923.0000 - fp: 7238.0000 - tn: 13146.0000 - fn: 1653.0000 - accuracy: 0.7829 - precision: 0.7233 - recall: 0.9197 - auc: 0.8379 - prc: 0.7730 - val_loss: 0.3961 - val_tp: 213379.0000 - val_fp: 556525.0000 - val_tn: 1057820.0000 - val_fn: 13044.0000 - val_accuracy: 0.6906 - val_precision: 0.2772 - val_recall: 0.9424 - val_auc: 0.8476 - val_prc: 0.3362
Epoch 38/100
20/20 [==============================] - 5s 252ms/step - loss: 0.4376 - tp: 18893.0000 - fp: 7049.0000 - tn: 13434.0000 - fn: 1584.0000 - accuracy: 0.7892 - precision: 0.7283 - recall: 0.9226 - auc: 0.8398 - prc: 0.7719 - val_loss: 0.3947 - val_tp: 211766.0000 - val_fp: 543077.0000 - val_tn: 1071268.0000 - val_fn: 14657.0000 - val_accuracy: 0.6970 - val_precision: 0.2805 - val_recall: 0.9353 - val_auc: 0.8476 - val_prc: 0.3367
Epoch 39/100
20/20 [==============================] - 5s 259ms/step - loss: 0.4416 - tp: 18735.0000 - fp: 7226.0000 - tn: 133

20/20 [==============================] - 5s 247ms/step - loss: 0.4359 - tp: 19502.0000 - fp: 7229.0000 - tn: 13089.0000 - fn: 1140.0000 - accuracy: 0.7957 - precision: 0.7296 - recall: 0.9448 - auc: 0.8434 - prc: 0.7829 - val_loss: 0.3980 - val_tp: 210079.0000 - val_fp: 530731.0000 - val_tn: 1083614.0000 - val_fn: 16344.0000 - val_accuracy: 0.7028 - val_precision: 0.2836 - val_recall: 0.9278 - val_auc: 0.8499 - val_prc: 0.3468
Epoch 56/100
20/20 [==============================] - 5s 249ms/step - loss: 0.4340 - tp: 19265.0000 - fp: 7305.0000 - tn: 13216.0000 - fn: 1174.0000 - accuracy: 0.7930 - precision: 0.7251 - recall: 0.9426 - auc: 0.8447 - prc: 0.7828 - val_loss: 0.4008 - val_tp: 213270.0000 - val_fp: 554670.0000 - val_tn: 1059675.0000 - val_fn: 13153.0000 - val_accuracy: 0.6915 - val_precision: 0.2777 - val_recall: 0.9419 - val_auc: 0.8502 - val_prc: 0.3474
Epoch 57/100
20/20 [==============================] - 5s 240ms/step - loss: 0.4303 - tp: 19691.0000 - fp: 7327.0000 - tn: 129

20/20 [==============================] - 5s 245ms/step - loss: 0.4311 - tp: 19383.0000 - fp: 7275.0000 - tn: 13190.0000 - fn: 1112.0000 - accuracy: 0.7952 - precision: 0.7271 - recall: 0.9457 - auc: 0.8469 - prc: 0.7857 - val_loss: 0.3980 - val_tp: 209636.0000 - val_fp: 527276.0000 - val_tn: 1087069.0000 - val_fn: 16787.0000 - val_accuracy: 0.7044 - val_precision: 0.2845 - val_recall: 0.9259 - val_auc: 0.8518 - val_prc: 0.3517
Epoch 74/100
20/20 [==============================] - 5s 241ms/step - loss: 0.4319 - tp: 19241.0000 - fp: 7256.0000 - tn: 13263.0000 - fn: 1200.0000 - accuracy: 0.7936 - precision: 0.7262 - recall: 0.9413 - auc: 0.8461 - prc: 0.7817 - val_loss: 0.3932 - val_tp: 209509.0000 - val_fp: 526436.0000 - val_tn: 1087909.0000 - val_fn: 16914.0000 - val_accuracy: 0.7048 - val_precision: 0.2847 - val_recall: 0.9253 - val_auc: 0.8519 - val_prc: 0.3517
Epoch 75/100
20/20 [==============================] - 5s 244ms/step - loss: 0.4343 - tp: 19050.0000 - fp: 7379.0000 - tn: 133

KeyboardInterrupt: 

In [41]:
y_pred_prob = resampled_model.predict(test_features)
roc_auc = roc_auc_score(test_labels, y_pred_prob)
print(f'ROC AUC: {roc_auc:.2f}')

71905/71905 [==============================] - 113s 2ms/step
ROC AUC: 0.85


In [46]:
y_pred_prob = resampled_model.predict(target)

239684/239684 [==============================] - 379s 2ms/step


In [47]:
submit = pd.read_csv("./playground-series-s4e7/sample_submission.csv")
pred = y_pred_prob
submit["Response"] = pred
submit.to_csv("submission_DNN.csv", index=False)
submit.head()

,id,Response
0,11504798,0.0
1,11504799,0.0
2,11504800,0.0
3,11504801,0.0
4,11504802,0.0
